<a href="https://colab.research.google.com/github/gulsumaltintas/Natural_Language_Processing/blob/main/Statiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Statiment Analysis

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import numpy as np
import pandas as pd
import re
import pickle
from nltk.corpus import stopwords
from sklearn.datasets import load_files

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
reviews = load_files('/gdrive/My Drive/txt_sentoken/')

In [ ]:
X,y = reviews.data, reviews.target # X = datanın özelliklerini saklayan değişken y = datanın sınıfını saklayan değişken

In [ ]:
#Bu işlemleri bir defa yapmamız yeterli
#Pickling işlemleri
with open("X.pickle","wb") as f:
  pickle.dump(X,f) #dosyanın içerisine kaydediyor.
with open("y.pickle","wb") as f:
  pickle.dump(y,f)

In [ ]:
#UnPickling işlemleri
#Belleğe dosya olarak aktarmış olduğumuz dosyaların içerisindeki verileri çekip tekrar bellekte tutulur hale getirme işlemi
X_in = open("X.pickle","rb")
y_in = open("y.pickle","rb")
X = pickle.load(X_in) # X içerisine alıyor X_in içerisindekileri (yerleştirme işlemi yapıyor)
y = pickle.load(y_in)

In [ ]:
#Proposition işlemleri
corpus = []
for i in range(0,2000):
  review = re.sub(r'\W',' ',str(X[i])) #değiştirme işlemi.İçerisinde alfanümerik karakter olamayanları ayıklama işlemi
  review = review.lower() #büyük harfleri küçük harfe dönüştürme
  #Bazı etiketler kalabiliyor bunların ayıklanmasıda gerçekleşebirlir.<br>gibi
  review = re.sub("^b\s+"," ",review) #birden fazla boşluk işlemleri
  review = re.sub("\s+[a-z]\s+]"," ",review)
  corpus.append(review)


In [ ]:
#Bag of Words modeli oluşturma
#histogramını kendimizin oluşturmasına gerek yok hazır bir kütüphane kullanabiliriz
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=2000, min_df=3, max_df=0.6, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()


In [ ]:
#TF-IDF modelinde transform aynı işlemi(countvectorizer) yapıyor
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X = transformer.fit_transform(X).toarray() #artık tf-ıdf değerleri yazmaya başladı
#arada hiçbir fark yok sadece countvec kullanılabilirde.

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = CountVectorizer(max_features=2000, min_df=3, max_df=0.6, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()


In [ ]:
#Ayırma işlemini gerçekleştiricez.Test için
#Eğitim işlemleri
from sklearn.model_selection import train_test_split
text_train, text_test, sent_train, sent_test = train_test_split(X,y,test_size = 0.2, random_state = 0)

from sklearn.linear_model import LogisticRegression
cl = LogisticRegression()
cl.fit(text_train, sent_train)

#Test işlemleri
sent_pred = cl.predict(text_test)

#Karmaşıklık matrisi çıkarmamız gerekiyor
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(sent_test, sent_pred)
print(cm)
#Modelimiz oluşturuldu




[[168  40]
 [ 27 165]]


In [ ]:
with open('cls.pickle',"wb") as f:
  pickle.dump(cl,f)


In [ ]:
#duygu analizi işlemimiz için ornek bir cümle
#derlem içerisindeki kelimeleri ve cümleleri tanıyarak orneğin negatif bir cümle mi yoksa pozitif bir cümle mi anlıyor
#Öğrenme olayını gerçekleştirmiş olduk
ornek = ["he is a nice person"]
ornek = vectorizer.transform(ornek).toarray()
sentiment = cl.predict(ornek)
print(sentiment)


[0]
